In [2]:
#Environment setup
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

#librarires
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

2022-11-20 16:26:54.789530: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
data_xmr = pd.read_csv("XMR-USD.csv")# Data loading


data_xmr.head()#data viewing

,Date,Open,High,Low,Close,Adj Close,Volume,key
0,2017-11-09,112.531998,123.404999,112.219002,120.779999,120.779999,86864600,xmr_usd
1,2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000,xmr_usd
2,2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000,xmr_usd
3,2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000,xmr_usd
4,2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000,xmr_usd


In [4]:
data_xmr['Close'].value_counts() 

69.162140     2
120.779999    1
205.749130    1
208.657867    1
204.832031    1
             ..
86.264908     1
88.423225     1
87.744942     1
99.962112     1
142.070938    1
Name: Close, Length: 1808, dtype: int64

In [5]:
df = data_xmr.loc[(data_xmr['key'] == 'xmr_usd')]
df.head(1810)

,Date,Open,High,Low,Close,Adj Close,Volume,key
0,2017-11-09,112.531998,123.404999,112.219002,120.779999,120.779999,86864600,xmr_usd
1,2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000,xmr_usd
2,2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000,xmr_usd
3,2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000,xmr_usd
4,2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000,xmr_usd
...,...,...,...,...,...,...,...,...
1804,2022-10-18,143.624588,148.039398,142.697784,146.093826,146.093826,118028731,xmr_usd
1805,2022-10-19,146.099319,148.462082,144.796875,145.044846,145.044846,81641779,xmr_usd
1806,2022-10-20,145.041626,148.424316,140.163910,141.207642,141.207642,86726962,xmr_usd
1807,2022-10-21,141.195435,143.249649,137.309967,140.729477,140.729477,80536580,xmr_usd


In [6]:
#date time collection
df = df.reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.loc[df['Date'] > pd.to_datetime('2017-06-28')]

In [7]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,key
0,2017-11-09,112.531998,123.404999,112.219002,120.779999,120.779999,86864600,xmr_usd
1,2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000,xmr_usd
2,2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000,xmr_usd
3,2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000,xmr_usd
4,2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000,xmr_usd


In [8]:
df = df[['Date','Open','Close','High','Low','Volume']]
df = df[['Close']]

In [9]:
dataset = df.values
dataset = dataset.astype('float32')

In [10]:
dataset

array([[120.78   ],
       [105.586  ],
       [119.616  ],
       ...,
       [141.20764],
       [140.72948],
       [142.07094]], dtype=float32)

In [11]:
#Rescaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
dataset_scale = scaler.fit_transform(dataset)

In [12]:
dataset_scale

array([[0.19479558],
       [0.16107407],
       [0.1922122 ],
       ...,
       [0.24013257],
       [0.23907134],
       [0.24204856]], dtype=float32)

In [13]:
#training set size
train_size = int(len(dataset_scale) * 0.709)
test_size = len(dataset_scale) - train_size
train, test = dataset_scale[0:train_size, :], dataset_scale[train_size:len(dataset), :]
print(len(train), len(test))

1282 527


In [14]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
  return np.array(dataX), np.array(dataY)

In [15]:
look_back = 20
trainX, trainY = create_dataset(train, look_back=look_back)
testX, testY = create_dataset(test, look_back=look_back)

In [16]:
testX

array([[0.8423271 , 0.77543116, 0.7943311 , ..., 0.5324532 , 0.5165964 ,
        0.57944345],
       [0.77543116, 0.7943311 , 0.681069  , ..., 0.5165964 , 0.57944345,
        0.61072963],
       [0.7943311 , 0.681069  , 0.68413126, ..., 0.57944345, 0.61072963,
        0.56473255],
       ...,
       [0.2559772 , 0.2536701 , 0.23938954, ..., 0.24447536, 0.2454688 ,
        0.25097695],
       [0.2536701 , 0.23938954, 0.2306107 , ..., 0.2454688 , 0.25097695,
        0.24864885],
       [0.23938954, 0.2306107 , 0.24127495, ..., 0.25097695, 0.24864885,
        0.24013257]], dtype=float32)

In [17]:
testY

array([0.61072963, 0.56473255, 0.5133415 , 0.5317323 , 0.47921848,
       0.49240625, 0.5355873 , 0.49347264, 0.46600914, 0.4659624 ,
       0.50643104, 0.5258305 , 0.5470484 , 0.5087037 , 0.54175144,
       0.5145316 , 0.52712935, 0.5139908 , 0.37807724, 0.3740151 ,
       0.3982781 , 0.42625195, 0.36285633, 0.37678036, 0.3954712 ,
       0.3982447 , 0.4086675 , 0.4200407 , 0.38518384, 0.39579538,
       0.40335825, 0.41465196, 0.40271956, 0.41974705, 0.40357527,
       0.3826924 , 0.39890665, 0.39182654, 0.39618665, 0.38573554,
       0.37849158, 0.37781903, 0.36218098, 0.3553605 , 0.3653154 ,
       0.36955106, 0.34574905, 0.33310112, 0.36264938, 0.36535892,
       0.37852502, 0.38984585, 0.40055   , 0.4117011 , 0.42854112,
       0.44784278, 0.46211982, 0.4656012 , 0.45976973, 0.45102793,
       0.44400662, 0.4265357 , 0.47041863, 0.48793668, 0.51051897,
       0.52814704, 0.4967969 , 0.5263876 , 0.51747835, 0.52279824,
       0.49733698, 0.53277946, 0.5288269 , 0.5309383 , 0.52125

In [18]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [19]:
testX

array([[[0.8423271 , 0.77543116, 0.7943311 , ..., 0.5324532 ,
         0.5165964 , 0.57944345]],

       [[0.77543116, 0.7943311 , 0.681069  , ..., 0.5165964 ,
         0.57944345, 0.61072963]],

       [[0.7943311 , 0.681069  , 0.68413126, ..., 0.57944345,
         0.61072963, 0.56473255]],

       ...,

       [[0.2559772 , 0.2536701 , 0.23938954, ..., 0.24447536,
         0.2454688 , 0.25097695]],

       [[0.2536701 , 0.23938954, 0.2306107 , ..., 0.2454688 ,
         0.25097695, 0.24864885]],

       [[0.23938954, 0.2306107 , 0.24127495, ..., 0.25097695,
         0.24864885, 0.24013257]]], dtype=float32)

In [20]:
#xmr model loading

model_xmr = tf.keras.models.load_model("xmr.h5")


#xmr model summary


In [21]:
model_xmr.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 4)                 400       
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 405
Trainable params: 405
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss= model_xmr.evaluate(testX,testY,batch_size=280,verbose=2)

2/2 - 1s - loss: 0.0011 - 1s/epoch - 750ms/step


In [34]:
print(loss)

0.0011460552923381329


In [44]:
#Model Prediction
scaler = MinMaxScaler(feature_range=(0, 1))
a = [[149.146301],[149.668274],[146.864075],[149.28302],[158.32048],[157.242599],[154.919769],[155.234894],[138.239548],[119.120728],[133.224411],[128.019562],[128.375656],[127.46772],[129.822495],[128.993073],[132.24736],[132.727921],[133.294037],[134.364105]]
a_scale= scaler.fit_transform(a)
#a = [65]
a = np.array(a_scale)

a = np.reshape(a, (1,1, 20))
#print(b.shape)

result = model_xmr.predict(a)
print(result[0])
rev_result=scaler.inverse_transform(result)
print(rev_result[0])

1/1 [==============================] - 0s 14ms/step
[0.2903516]
[130.50244]
